In [1]:
%load_ext autoreload
%autoreload 2

In [97]:
from core import solve, solve_sample, prod, read_sample_input, xor

import toolz.curried as tz
import operator
import time

import re

from dataclasses import dataclass

# day 1

In [3]:
sample_1 = """
1
1721
979
366
299
675
1456
1990
"""

In [4]:
@tz.curry
def two_elems_sum_to_target(target, sequence, is_sorted=False):
    """
    Find two elements in a sequence that sum to the target
    """
    def recurse(target, seq):
        if len(seq) < 2:
            return None
        first, last = seq[0], seq[-1]
        if first + last == target:
            return first, last
        if first + last < target:
            return recurse(target, seq[1:])
        return recurse(target, seq[:-1])

    return recurse(target, sequence if is_sorted else sorted(sequence))

In [8]:
def solve_1a(inp):
    return tz.pipe(
        inp, 
        tz.map(int), 
        list, 
        two_elems_sum_to_target(2020), 
        prod)

In [11]:
solve_sample(solve_1a, sample_1), solve(solve_1a, 1)

(514579, 972576)

In [22]:
@tz.curry
def three_elems_sum_to_target(target, sequence):
    sorted_sequence = sorted(sequence)
    for idx, elem in enumerate(sorted_sequence):
        result = two_elems_sum_to_target(
            target - elem, 
            sorted_sequence[:idx] + sorted_sequence[(idx+1):], 
            is_sorted=True
        )
        if result:
            return elem, *result


In [19]:
def solve_1b(inp):
    return tz.pipe(
        inp, 
        tz.map(int), 
        list, 
        three_elems_sum_to_target(2020), 
        prod)

In [20]:
solve_sample(solve_1b, sample_1), solve(solve_1b, 1)

(241861950, 199300880)

# day 2

In [29]:
sample_2 = """
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
"""

In [101]:
password_pattern = re.compile("(\d+)-(\d+)\s(\w):\s(\w+)")

@dataclass
class PasswordInput:
    first: int
    second: int
    char: str
    password: str

def re_to_passwordinput(match):
    first, second, char, password = match[0]
    return PasswordInput(int(first), int(second), char, password)

def password_is_valid(password_input):
    return password_input.first <= password_input.password.count(password_input.char) <= password_input.second

In [102]:
def solve_2a(inp):
    return tz.pipe(
        inp,
        tz.map(password_pattern.findall),
        tz.map(re_to_passwordinput),
        tz.filter(password_is_valid),
        tz.count
    )

In [103]:
solve_sample(solve_2a, sample_2), solve(solve_2a, 2)

(2, 493)

In [104]:
def password_is_valid_b(password_input):
    first_valid = password_input.password[password_input.first - 1] == password_input.char
    second_valid = password_input.password[password_input.second - 1] == password_input.char
    return xor(first_valid, second_valid)


In [105]:
def solve_2b(inp):
    return tz.pipe(
        inp,
        tz.map(password_pattern.findall),
        tz.map(re_to_passwordinput),
        tz.filter(password_is_valid_b),
        tz.count
    )

In [106]:
solve_sample(solve_2b, sample_2), solve(solve_2b, 2)

(1, 593)

# day 3

In [107]:
sample_3 = """
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
"""

In [278]:
def rot_index(idx, arr):
    return idx % len(arr)

def rot_value_at(idx, arr):
    return arr[rot_index(idx, arr)]


In [296]:
@tz.curry
def count_trees_and_advance(right, state, section):
    location, trees = state
    trees += (rot_value_at(location, section) == '#')
    location += right
    return (location, trees)

In [ ]:
def toboggan_travel(forest, down=1, right=1):
    return tz.pipe(
        forest,
        tz.take_nth(down),
        lambda forest: tz.reduce(count_trees_and_advance(right), forest, (0, 0)),
        tz.last
    )

In [305]:
def solve_3a(inp):
    return tz.pipe(
        inp,
        tz.map(lambda s: s.strip()),
        lambda forest: tz.reduce(count_trees_and_advance(3), forest, (0, 0)),
        tz.last
    )

In [306]:
solve_sample(solve_3a, sample_3), solve(solve_3a, 3)

(7, 223)

In [313]:
forest

[]

In [321]:
forest = tz.pipe(
    read_sample_input(sample_3), 
    tz.map(lambda s: s.strip()),
    list
)

tz.pipe(
    [1, 3, 5, 7], 
    tz.map(
    lambda steps:
        tz.pipe(
            lambda forest: tz.reduce(count_trees_and_advance(steps), forest, (0, 0)),
            tz.last
        )),
    list
)

TypeError: 'function' object is not iterable

In [323]:
tz.pipe(
    [1, 3, 5, 7], 
    tz.map(
    lambda x:
        x**2),
    list
)

[1, 9, 25, 49]